## The String Widgets

+ HTML
+ HTMLMath
+ Label
+ Password
+ Text
+ TextArea

These widgets are used to display data conventionally represented as strings.

In [1]:
{-# LANGUAGE OverloadedStrings #-}
{-# LANGUAGE FlexibleContexts #-}
import IHaskell.Display.Widgets

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:

In [18]:
-- Constructors
html <- mkHTML
htmlMath <- mkHTMLMath
text <- mkText
area <- mkTextArea
pass <- mkPassword

These widgets have a `Text` payload, represented by the `StringValue` field.

### HTML and Latex

The `HTML` widget displays `Text` as rich formatted *HTML*, but the `HTMLMath` widget can display *LaTeX* as well.

In [10]:
-- Display the widgets
html

In [12]:
-- Set some html string
setField html StringValue "<b>Hello</b> <i>World!</i>"

In [13]:
-- Set some latex string
setField htmlMath StringValue "$$\\sum_{i=0}^{i < n} i =\\frac{n\\cdot(n+1)}{2}$$"
htmlMath

###  `TextWidget`, `Password` & `TextArea`

These widgets allow you to obtain text input. First, let's see what they look like:

In [19]:
text
pass
area

The first one has been created to input a word, but the second one is bigger and resizeable, making it better to big text input. The `TextWidget` and `TextArea` also have a `Placeholder` property, which represents the text displayed when the user has not made any input yet.

> **NOTE ABOUT PASSWORD WIDGET**: This widget **is not** a **secure** way to collect sensitive information.
> - The contents of the `Password` widget are transmitted unencrypted between the frontend and the kernel
> - If you save the notebook, the contents of the widget are stored as plaintext

In [25]:
setField text Placeholder "Enter your text here..."
setField pass Placeholder "Password"
setField area Placeholder "Parsed output will appear here..."

In [27]:
getField text StringValue
getField pass StringValue
getField area StringValue

""

"agsagsa"

""

The widgets also accept input. The `StringValue` of the widget is automatically updated on every change to the widget. Additionally, the `TextWidget` also has a `SubmitHandler` which is triggered on hitting the return/enter key.

Below we set up the `TextWidget` and `TextArea` for parsing phone numbers using parsec. The `TextWidget` is used to recieve input, and the `TextArea` is used to display output.

In [28]:
-- Import parsec and other required libraries

import Text.Parsec
import Text.Parsec.String
import Data.Text (pack, unpack)
import Control.Applicative ((<$>))

Now, we can write some parsers:

In [29]:
-- Parse a single digit
digit :: Parser Char
digit = oneOf ['0'..'9']

-- Parse a multi-digit number.
number :: Parser Integer
number = do
  digits <- many1 digit -- At least one digit
  return (read digits)  -- Convert [Char] to Integer
  
-- Parse a country code, starting with a +.
countryCode :: Parser Integer
countryCode = do
  char '+'
  number
  
-- Parse an area code, optionally with parentheses.
areaCode :: Parser Integer
areaCode = choice [withParens, withoutParens]
  where
    withParens = between (char '(') (char ')') withoutParens
    withoutParens = number
  
-- Simple data type representing a phone number.
-- Real phone numbers are much more complex!
data PhoneNumber = PhoneNumber {
    phoneCountryCode :: Maybe Integer,
    phoneNumbers :: [Integer]
  } deriving (Eq, Show)
  
phoneNumber :: Parser PhoneNumber
phoneNumber = do
  -- Try to parse a country code. If it doesn't work, it's Nothing.
  c <- optionMaybe countryCode
  optional separator
  a1 <- areaCode
  separator -- Separator required after area code
  a2 <- number
  separator -- Separator required before last group of digits
  a3 <- number
  return (PhoneNumber c [a1, a2, a3])
  
  where
    separator = oneOf " -"

Now, we set the `TextWidget`'s change handler to parse the input, and write the output to the `TextArea`.

In [30]:
setField text ChangeHandler $ do
  input <- unpack <$> getField text StringValue
  str <- case parse phoneNumber "<text widget>" input of
             Left error -> return (show error)
             Right x -> return (show x)
  setField area StringValue (pack str)

The `TextArea` doesn't have a `SubmitHandler`, but does have a `ChangeHandler`. It is best used to display large amounts of text.

We can re-display the widgets (nobody likes to scroll needlessly). Now you can input a telephone number on the text widget, and it should be parsed and displayed to the text area.

In [31]:
text
area

### The `Password` widget

It works the same as the `Text` widget, but it hides the text from the user. Can you guess the password and make the valid widget green?

In [36]:
valid <- mkValid
pwd <- mkPassword

setField pwd ChangeHandler $ getField pwd StringValue >>= setField valid BoolValue . ("1234"==)

pwd
valid